In [15]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="dataset.jsonl")
print(dataset)


FileNotFoundError: Unable to find '/home/force-4/Desktop/model_finetuning/dataset.jsonl'

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "mistralai/Mistral-7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",
    use_auth_token=True
)


/home/force-4/Desktop/model_finetuning/env/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:1001: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/force-4/Desktop/model_finetuning/env/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 2/2 [00:13<00:00,  6.79s/it]


In [4]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
model.print_trainable_parameters()


trainable params: 6,815,744 || all params: 7,248,547,840 || trainable%: 0.0940


In [5]:
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq

training_args = TrainingArguments(
    output_dir="./mistral-finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=3,
    fp16=True,
    save_strategy="epoch",
    logging_dir="./logs",
    report_to="none"
)

data_collator = DataCollatorForSeq2Seq(tokenizer, padding=True)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()


/tmp/ipykernel_40543/3910629872.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


KeyError: "Invalid key: 0. Please first select a split. For example: `my_dataset_dictionary['train'][0]`. Available splits: ['train']"

In [8]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="train.jsonl")
print(dataset)
print(dataset["train"][0])


DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 10
    })
})
{'messages': [{'role': 'system', 'content': 'You are Vishnu CK, an AI developer, data scientist, and computer science expert.'}, {'role': 'user', 'content': 'Who are you?'}, {'role': 'assistant', 'content': 'I am Vishnu CK, an AI developer with expertise in Python, machine learning, and computer vision systems, especially CCTV-based AI applications.'}]}


In [16]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="data.json")
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 7
    })
})


In [18]:
def format_messages(example):
    text = ""
    for m in example["messages"]:
        text += f"{m['role']}: {m['content']}\n"
    return {"text": text}

dataset = dataset.map(format_messages)


Map: 100%|██████████| 7/7 [00:00<00:00, 1773.38 examples/s]


In [19]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,  # QLoRA
    device_map="auto"
)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 3/3 [00:11<00:00,  3.81s/it]


In [ ]:
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments, Trainer

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj","v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    warmup_steps=10,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=50,
    save_total_limit=2,
    fp16=True,
    optim="paged_adamw_8bit"
)

def tokenize(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=["messages","text"])

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
)
trainer.train()


Map: 100%|██████████| 7/7 [00:00<00:00, 1067.91 examples/s]


ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.

: 

In [1]:
from datasets import Dataset, DatasetDict

# Example: Your dataset
data = [
    {"messages": [
        {"role": "system", "content": "You are Vishnu CK, an AI developer, data scientist, and computer science expert."},
        {"role": "user", "content": "Who are you?"},
        {"role": "assistant", "content": "I am Vishnu CK, an AI developer with expertise in Python, machine learning, and computer vision systems, especially CCTV-based AI applications."}
    ]}
]

dataset = Dataset.from_list(data)
dataset_dict = DatasetDict({"train": dataset})

print(dataset_dict)


/home/force-4/Desktop/model_finetuning/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 1
    })
})


In [2]:
def format_chat(example):
    messages = example["messages"]
    text = ""
    for msg in messages:
        text += f"{msg['role'].upper()}: {msg['content']}\n"
    return {"text": text}

formatted_dataset = dataset_dict.map(format_chat)
print(formatted_dataset["train"][0])


Map: 100%|██████████| 1/1 [00:00<00:00, 326.20 examples/s]

{'messages': [{'content': 'You are Vishnu CK, an AI developer, data scientist, and computer science expert.', 'role': 'system'}, {'content': 'Who are you?', 'role': 'user'}, {'content': 'I am Vishnu CK, an AI developer with expertise in Python, machine learning, and computer vision systems, especially CCTV-based AI applications.', 'role': 'assistant'}], 'text': 'SYSTEM: You are Vishnu CK, an AI developer, data scientist, and computer science expert.\nUSER: Who are you?\nASSISTANT: I am Vishnu CK, an AI developer with expertise in Python, machine learning, and computer vision systems, especially CCTV-based AI applications.\n'}


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "mistralai/Mistral-7B-v0.1"  # or LLaMA

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,   # saves GPU memory
    device_map="auto"
)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.61s/it]


In [6]:
from transformers import AutoTokenizer

model_name = "mistralai/Mistral-7B-v0.1"  # or your model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Add pad_token if missing
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    # OR equivalently you can do this:
    # tokenizer.pad_token = tokenizer.eos_token


In [7]:
def tokenize(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",   # pads to max_length
        truncation=True,
        max_length=512
    )

tokenized_dataset = formatted_dataset.map(tokenize, batched=True)


Map: 100%|██████████| 1/1 [00:00<00:00, 196.31 examples/s]


In [8]:
from peft import LoraConfig, get_peft_model
import torch

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./finetuned_model",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    save_strategy="epoch",
    logging_dir="./logs",
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"]
)

trainer.train()


/pytorch/aten/src/ATen/native/cuda/IndexKernelUtils.cu:16: vectorized_gather_kernel: block: [0,0,0], thread: [96,0,0] Assertion `ind >=0 && ind < ind_dim_size && "vectorized gather kernel index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/IndexKernelUtils.cu:16: vectorized_gather_kernel: block: [0,0,0], thread: [97,0,0] Assertion `ind >=0 && ind < ind_dim_size && "vectorized gather kernel index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/IndexKernelUtils.cu:16: vectorized_gather_kernel: block: [0,0,0], thread: [98,0,0] Assertion `ind >=0 && ind < ind_dim_size && "vectorized gather kernel index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/IndexKernelUtils.cu:16: vectorized_gather_kernel: block: [0,0,0], thread: [99,0,0] Assertion `ind >=0 && ind < ind_dim_size && "vectorized gather kernel index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/IndexKernelUtils.cu:16: vectorized_gather_kernel: block: [0,0,0], thread: [100,0,0] Assertion

AcceleratorError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


: 